In [1]:
import random

# Arguments
REWARD = -1 # constant reward for non-terminal states
DISCOUNT = 0.99
MAX_ERROR = 10**(-2)

# Set up the initial environment
NUM_ACTIONS = 4
ACTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)] # Down, Left, Up, Right
NUM_ROW = 5
NUM_COL = 6
WALL = (4,8,10,14,16,27)
U = [[0, 0, 0, 0, -float("inf"), 1], 
    [0, 0, -float("inf"), 0, -float("inf"), 0], 
    [0, 0, -float("inf"), 0, -float("inf"), 0], 
    [0, 0, 0, 0, 0, 0], 
    [0, 0, 0, -float("inf"), 0, 0]]
policy = [[random.randint(0, 3) for j in range(NUM_COL)] for i in range(NUM_ROW)] # construct a random policy

# Visualization
def printEnvironment(arr, policy=False):
    res = ""
    for r in range(NUM_ROW):
        res += "|"
        for c in range(NUM_COL):
            if r * NUM_COL + c in WALL:
                val = "WALL"
            elif r == 0 and c == 5:
                val = "+1"
            else:
                val = ["Down", "Left", "Up", "Right"][arr[r][c]]
            res += " " + val[:5].ljust(5) + " |" # format
        res += "\n"
    print(res)

# Get the utility of the state reached by performing the given action from the given state
def getU(U, r, c, action):
    dr, dc = ACTIONS[action]
    newR, newC = r+dr, c+dc
    if newR < 0 or newC < 0 or newR >= NUM_ROW or newC >= NUM_COL or (newR * NUM_COL + newC in WALL): # collide with the boundary or the wall
        return U[r][c]
    else:
        return U[newR][newC]

# Calculate the utility of a state given an action
def calculateU(U, r, c, action):
    u = REWARD
    u += 0.1 * DISCOUNT * getU(U, r, c, (action-1)%4)
    u += 0.8 * DISCOUNT * getU(U, r, c, action)
    u += 0.1 * DISCOUNT * getU(U, r, c, (action+1)%4)
    return u

# Perform some simplified value iteration steps to get an approximation of the utilities
def policyEvaluation(policy, U):
    while True:
        nextU = [[0, 0, 0, 0, -float("inf"), 1], 
                 [0, 0, -float("inf"), 0, -float("inf"), 0], 
                 [0, 0, -float("inf"), 0, -float("inf"), 0], 
                 [0, 0, 0, 0, 0, 0], 
                 [0, 0, 0, -float("inf"), 0, 0]]
        error = 0
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r == 0 and c == 5) or (r * NUM_COL + c in WALL):
                    continue
                nextU[r][c] = calculateU(U, r, c, policy[r][c]) # simplified Bellman update
                error = max(error, abs(nextU[r][c]-U[r][c]))
        U = nextU
        if error < MAX_ERROR * (1-DISCOUNT) / DISCOUNT:
            break
    return U

def policyIteration(policy, U):
    print("During the policy iteration:\n")
    while True:
        U = policyEvaluation(policy, U)
        unchanged = True
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r == 0 and c == 5) or (r * NUM_COL + c in WALL):
                    continue
                maxAction, maxU = None, -float("inf")
                for action in range(NUM_ACTIONS):
                    u = calculateU(U, r, c, action)
                    if u > maxU:
                        maxAction, maxU = action, u
                if maxU > calculateU(U, r, c, policy[r][c]):
                    policy[r][c] = maxAction # the action that maximizes the utility
                    unchanged = False
        if unchanged:
            break
        printEnvironment(policy)
    return policy

# Print the initial environment
print("The initial random policy is:\n")
printEnvironment(policy)

# Policy iteration
policy = policyIteration(policy, U)

# Print the optimal policy
print("The optimal policy is:\n")
printEnvironment(policy)

The initial random policy is:

| Down  | Down  | Down  | Right | WALL  | +1    |
| Right | Left  | WALL  | Down  | WALL  | Right |
| Right | Up    | WALL  | Up    | WALL  | Up    |
| Left  | Down  | Right | Down  | Left  | Right |
| Up    | Left  | Right | WALL  | Right | Right |

During the policy iteration:

| Down  | Left  | Left  | Left  | WALL  | +1    |
| Down  | Down  | WALL  | Down  | WALL  | Up    |
| Down  | Down  | WALL  | Down  | WALL  | Up    |
| Right | Right | Right | Down  | Right | Up    |
| Right | Right | Right | WALL  | Right | Up    |

| Down  | Down  | Left  | Down  | WALL  | +1    |
| Right | Down  | WALL  | Down  | WALL  | Up    |
| Right | Down  | WALL  | Down  | WALL  | Up    |
| Right | Right | Right | Right | Right | Up    |
| Up    | Up    | Up    | WALL  | Right | Up    |

| Down  | Down  | Right | Down  | WALL  | +1    |
| Down  | Down  | WALL  | Down  | WALL  | Up    |
| Down  | Down  | WALL  | Down  | WALL  | Up    |
| Right | Right | Right | Right | Ri